In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn

In [2]:
import geopandas as gpd
import pandas as pd

## Load daily temperature 

In [3]:
years = [year for year in range(1960, 2020)]

In [4]:
pattern = []
for year in years:
    temp_year = ddf.read_csv(r'/global/cfs/cdirs/m1532/Projects_MVP/geospatial/PRISM_Data/PRISM_daily_county_level/prism_daily_county_level_' + str(year) + '.csv', dtype = {'year': int, 'fips': str}).compute().drop(columns={'Unnamed: 0'})
    pattern.append(temp_year)

In [5]:
temp = pd.concat(pattern)
temp.head()

,fips,date,tMean,tMin,tMax,prec,year
0,01001,1960-01-01,5.417989,2.635663,8.200316,2.279021,1960
1,01001,1960-01-02,7.005736,4.201073,9.810398,10.412733,1960
2,01001,1960-01-03,9.061477,5.235675,12.887278,16.011329,1960
3,01001,1960-01-04,5.334175,-0.744570,11.412921,0.216112,1960
4,01001,1960-01-05,6.495222,3.006032,9.984412,0.274311,1960


In [6]:
temp['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019])

In [7]:
import datetime

In [8]:
def convertTime(time):
    return datetime.strptime(time, '%Y-%m-%d')

In [9]:
def getMonth(date):
    return(date.strftime('%b'))

In [10]:
temp['date'] = pd.to_datetime(temp['date'])
temp['month'] = temp['date'].apply(getMonth)

In [11]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}

In [12]:
temp['month'] = temp['month'].replace(month_replace)
temp.head()

,fips,date,tMean,tMin,tMax,prec,year,month
0,01001,1960-01-01,5.417989,2.635663,8.200316,2.279021,1960,1
1,01001,1960-01-02,7.005736,4.201073,9.810398,10.412733,1960,1
2,01001,1960-01-03,9.061477,5.235675,12.887278,16.011329,1960,1
3,01001,1960-01-04,5.334175,-0.744570,11.412921,0.216112,1960,1
4,01001,1960-01-05,6.495222,3.006032,9.984412,0.274311,1960,1


In [21]:
climate = temp[['year', 'month', 'date', 'fips', 'tMean', 'tMin', 'tMax']]
climate.head()

,year,month,date,fips,tMean,tMin,tMax
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412


In [22]:
arizona = climate[(climate['fips'] == '04013') & (climate['month'] == 7)]
arizona

,year,month,date,fips,tMean,tMin,tMax
27266,1960,7,1960-07-01,04013,32.574107,22.326286,42.821929
27267,1960,7,1960-07-02,04013,34.630369,26.586453,42.674284
27268,1960,7,1960-07-03,04013,33.936788,26.412528,41.461047
27269,1960,7,1960-07-04,04013,31.660556,22.925237,40.395875
27270,1960,7,1960-07-05,04013,30.997479,22.093775,39.901183
...,...,...,...,...,...,...,...
27217,2019,7,2019-07-27,04013,35.028518,27.783607,42.273429
27218,2019,7,2019-07-28,04013,35.377315,28.138999,42.615630
27219,2019,7,2019-07-29,04013,35.150217,28.151650,42.148783
27220,2019,7,2019-07-30,04013,33.723849,26.492046,40.955653


### After data imputation, 97% of the counties have daily apparent tempeature data from 1960 to 2020

In [23]:
fips_numbers = []
for year in years:
    dict_year = {}
    df_year = climate[climate['year'] == year]
    df_new = df_year.dropna(subset = 'tMean')
    numbers = len(df_new['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [24]:
fips_numbers

[{1960: 3105},
 {1961: 3105},
 {1962: 3105},
 {1963: 3105},
 {1964: 3105},
 {1965: 3105},
 {1966: 3105},
 {1967: 3105},
 {1968: 3105},
 {1969: 3105},
 {1970: 3105},
 {1971: 3105},
 {1972: 3105},
 {1973: 3105},
 {1974: 3105},
 {1975: 3105},
 {1976: 3105},
 {1977: 3105},
 {1978: 3105},
 {1979: 3105},
 {1980: 3105},
 {1981: 3105},
 {1982: 3105},
 {1983: 3105},
 {1984: 3105},
 {1985: 3105},
 {1986: 3105},
 {1987: 3105},
 {1988: 3105},
 {1989: 3105},
 {1990: 3105},
 {1991: 3105},
 {1992: 3105},
 {1993: 3105},
 {1994: 3105},
 {1995: 3105},
 {1996: 3105},
 {1997: 3105},
 {1998: 3105},
 {1999: 3105},
 {2000: 3105},
 {2001: 3105},
 {2002: 3105},
 {2003: 3105},
 {2004: 3105},
 {2005: 3105},
 {2006: 3105},
 {2007: 3105},
 {2008: 3105},
 {2009: 3105},
 {2010: 3105},
 {2011: 3105},
 {2012: 3105},
 {2013: 3105},
 {2014: 3105},
 {2015: 3105},
 {2016: 3105},
 {2017: 3105},
 {2018: 3105},
 {2019: 3105}]

In [25]:
climate = climate.sort_values(by=['fips', 'year', 'month', 'date'])
climate.head()

,year,month,date,fips,tMean,tMin,tMax
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412


In [26]:
normal_period = climate[(climate['year'] >= 1961) & (climate['year'] <= 1990)]
normal_avg_temp = normal_period.groupby('fips')['tMax'].mean().reset_index()
normal_avg_temp.rename(columns={'tMax': 'avg_max_temp'}, inplace=True)
normal_avg_temp

,fips,avg_max_temp
0,01001,24.381066
1,01003,25.348830
2,01005,24.734360
3,01007,23.907312
4,01009,22.143142
...,...,...
3100,56037,12.825295
3101,56039,8.322003
3102,56041,12.441291
3103,56043,14.578120


In [27]:
climate = climate.merge(normal_avg_temp, on='fips')

In [28]:
climate['exceedance'] = (climate['tMax'] >= climate['avg_max_temp'] + 5)
climate.head()

,year,month,date,fips,tMean,tMin,tMax,avg_max_temp,exceedance
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,24.381066,False
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,24.381066,False
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,24.381066,False
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,24.381066,False
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,24.381066,False


In [43]:
example = climate[(climate['fips'] == '01001') & (climate['date'] >= '1960-06-11') & ((climate['date'] <= '1960-08-11'))]
example.head(50)

,year,month,date,fips,tMean,tMin,tMax,avg_max_temp,exceedance,block,state
162,1960,6,1960-06-11,01001,23.185523,15.693614,30.677431,24.381066,True,10,01
163,1960,6,1960-06-12,01001,24.327362,17.151675,31.503048,24.381066,True,10,01
164,1960,6,1960-06-13,01001,23.859629,16.423425,31.295833,24.381066,True,10,01
165,1960,6,1960-06-14,01001,26.104988,20.220610,31.989366,24.381066,True,10,01
166,1960,6,1960-06-15,01001,24.306202,16.745175,31.867229,24.381066,True,10,01
167,1960,6,1960-06-16,01001,25.775640,17.580736,33.970545,24.381066,True,10,01
168,1960,6,1960-06-17,01001,25.665286,20.369457,30.961115,24.381066,True,10,01
169,1960,6,1960-06-18,01001,23.140213,15.584670,30.695756,24.381066,True,10,01
170,1960,6,1960-06-19,01001,25.318903,18.345567,32.292240,24.381066,True,10,01
171,1960,6,1960-06-20,01001,26.457353,20.245531,32.669174,24.381066,True,10,01


In [47]:
example2 = climate[(climate['fips'] == '12001') & (climate['date'] >= '1960-06-29') & ((climate['date'] <= '1960-07-07'))]
example2

,year,month,date,fips,tMean,tMin,tMax,avg_max_temp,exceedance,block,state
6355530,1960,6,1960-06-29,12001,26.986585,20.500248,33.472921,27.370581,True,407636,12
6355531,1960,6,1960-06-30,12001,28.152707,20.989113,35.316300,27.370581,True,407636,12
6355532,1960,7,1960-07-01,12001,28.268658,21.813299,34.724017,27.370581,True,407636,12
6355533,1960,7,1960-07-02,12001,28.260334,22.213501,34.307166,27.370581,True,407636,12
6355534,1960,7,1960-07-03,12001,28.829784,22.504863,35.154704,27.370581,True,407636,12
6355535,1960,7,1960-07-04,12001,27.811393,21.708743,33.914043,27.370581,True,407636,12
6355536,1960,7,1960-07-05,12001,27.510530,21.699561,33.321499,27.370581,True,407636,12
6355537,1960,7,1960-07-06,12001,28.423515,22.635563,34.211467,27.370581,True,407636,12
6355538,1960,7,1960-07-07,12001,27.856546,22.063322,33.649771,27.370581,True,407636,12


In [29]:
# Group by county and identify consecutive exceedance days
climate['block'] = (climate['exceedance'] != climate['exceedance'].shift(1)).cumsum()
df_heatwaves = climate.groupby(['fips', 'block']).agg(
    start_date=('date', 'min'),
    end_date=('date', 'max'),
    exceedance_days=('exceedance', 'sum')
)

# Filter blocks where exceedance days are 5 or more
heatwaves = df_heatwaves[df_heatwaves['exceedance_days'] > 5].reset_index()

In [30]:
heatwaves

,fips,block,start_date,end_date,exceedance_days
0,01001,4,1960-05-17,1960-05-26,10
1,01001,8,1960-06-04,1960-06-09,6
2,01001,10,1960-06-11,1960-08-11,62
3,01001,12,1960-08-13,1960-09-13,32
4,01001,14,1960-09-18,1960-09-23,6
...,...,...,...,...,...
832070,56045,4800140,2018-08-22,2018-09-19,29
832071,56045,4800156,2019-05-30,2019-06-08,10
832072,56045,4800158,2019-06-12,2019-06-20,9
832073,56045,4800160,2019-06-24,2019-09-10,79


In [46]:
heatwaves['state'] = heatwaves['fips'].apply(findState)
heatwaves_florida = heatwaves[heatwaves['state'] == '12']
heatwaves_florida

,fips,block,start_date,end_date,exceedance_days,year,month,state
77041,12001,407622,1960-05-17,1960-05-26,10,1960,5,12
77042,12001,407636,1960-06-29,1960-07-07,9,1960,6,12
77043,12001,407640,1960-07-12,1960-07-17,6,1960,7,12
77044,12001,407642,1960-07-19,1960-07-27,9,1960,7,12
77045,12001,407648,1960-08-04,1960-08-17,14,1960,8,12
...,...,...,...,...,...,...,...,...
91790,12133,499938,2019-05-17,2019-06-06,21,2019,5,12
91791,12133,499942,2019-06-14,2019-07-24,41,2019,6,12
91792,12133,499944,2019-07-26,2019-08-20,26,2019,7,12
91793,12133,499946,2019-08-22,2019-08-27,6,2019,8,12


In [31]:
# Extract year from start_date
heatwaves['year'] = heatwaves['start_date'].dt.year
heatwaves['month'] = heatwaves['start_date'].dt.month

# Count heatwaves per year per county
yearly_heatwaves = heatwaves.groupby(['fips', 'year', 'month']).size().reset_index(name='heatwave_count')

yearly_heatwaves

,fips,year,month,heatwave_count
0,01001,1960,5,1
1,01001,1960,6,2
2,01001,1960,8,1
3,01001,1960,9,1
4,01001,1961,5,1
...,...,...,...,...
639456,56045,2018,6,1
639457,56045,2018,8,1
639458,56045,2019,5,1
639459,56045,2019,6,2


In [32]:
yearly_heatwaves['heatwave_count'].describe()

count    639461.000000
mean          1.301213
std           0.518636
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           4.000000
Name: heatwave_count, dtype: float64

In [33]:
yearly_heatwaves['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019], dtype=int32)

In [34]:
def findState(code):
    return code[:2]

In [35]:
yearly_heatwaves['state'] = yearly_heatwaves['fips'].apply(findState)
yearly_heatwaves.head()

,fips,year,month,heatwave_count,state
0,01001,1960,5,1,01
1,01001,1960,6,2,01
2,01001,1960,8,1,01
3,01001,1960,9,1,01
4,01001,1961,5,1,01


In [44]:
florida = yearly_heatwaves[yearly_heatwaves['state'] == '12']
florida

,fips,year,month,heatwave_count,state
59597,12001,1960,5,1,12
59598,12001,1960,6,1,12
59599,12001,1960,7,2,12
59600,12001,1960,8,1,12
59601,12001,1960,9,1,12
...,...,...,...,...,...
70501,12133,2018,9,2,12
70502,12133,2019,5,1,12
70503,12133,2019,6,1,12
70504,12133,2019,7,1,12


In [41]:
yearly_heatwaves.to_csv('heatwave_count_county_level.csv')

In [36]:
arizona = yearly_heatwaves[yearly_heatwaves['state'] == '04']
arizona

,fips,year,month,heatwave_count,state
13984,04001,1960,5,2,04
13985,04001,1960,9,1,04
13986,04001,1961,5,1,04
13987,04001,1961,9,1,04
13988,04001,1962,4,1,04
...,...,...,...,...,...
16778,04027,2017,6,1,04
16779,04027,2017,9,1,04
16780,04027,2018,5,2,04
16781,04027,2018,6,1,04


In [37]:
arizona_zero = arizona[arizona['heatwave_count'] == 0]
arizona_nonzero = arizona[arizona['heatwave_count'] != 0]

In [38]:
arizona_nonzero

,fips,year,month,heatwave_count,state
13984,04001,1960,5,2,04
13985,04001,1960,9,1,04
13986,04001,1961,5,1,04
13987,04001,1961,9,1,04
13988,04001,1962,4,1,04
...,...,...,...,...,...
16778,04027,2017,6,1,04
16779,04027,2017,9,1,04
16780,04027,2018,5,2,04
16781,04027,2018,6,1,04


In [39]:
arizona_zero.shape

(0, 5)

In [40]:
climate['state'] = climate['fips'].apply(findState)
climate.head()

,year,month,date,fips,tMean,tMin,tMax,avg_max_temp,exceedance,block,state
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,24.381066,False,1,01
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,24.381066,False,1,01
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,24.381066,False,1,01
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,24.381066,False,1,01
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,24.381066,False,1,01


In [89]:
arizona = climate[climate['state'] == '04']
arizona

,county,fips,date,AT_mean,AT_min,AT_max,year,month,day,state
78,"Apache County, AZ",04001,1960-01-01,-9.530005,-15.646542,-1.483143,1960,1,1,04
3311,"Apache County, AZ",04001,1960-01-02,-9.530005,-15.646542,-1.483143,1960,1,2,04
6544,"Apache County, AZ",04001,1960-01-03,-9.530005,-15.646542,-1.483143,1960,1,3,04
9777,"Apache County, AZ",04001,1960-01-04,-9.530005,-15.646542,-1.483143,1960,1,4,04
13010,"Apache County, AZ",04001,1960-01-05,-9.530005,-15.646542,-1.483143,1960,1,5,04
...,...,...,...,...,...,...,...,...,...,...
1170341,"Yuma County, AZ",04027,2020-12-27,4.994311,-3.125984,13.745060,2020,12,27,04
1173574,"Yuma County, AZ",04027,2020-12-28,4.994311,-3.125984,13.745060,2020,12,28,04
1176807,"Yuma County, AZ",04027,2020-12-29,4.994311,-3.125984,13.745060,2020,12,29,04
1180040,"Yuma County, AZ",04027,2020-12-30,4.994311,-3.125984,13.745060,2020,12,30,04


In [90]:
arizona_2011 = arizona[(arizona['year'] == 2011) & (arizona['month'] == 7) & (arizona['fips'] == '04013')]
arizona_2011

,county,fips,date,AT_mean,AT_min,AT_max,year,month,day,state
587008,"Maricopa County, AZ",04013,2011-07-01,6.910483,-2.588459,17.277122,2011,7,1,04
590241,"Maricopa County, AZ",04013,2011-07-02,6.910483,-2.588459,17.277122,2011,7,2,04
593474,"Maricopa County, AZ",04013,2011-07-03,6.910483,-2.588459,17.277122,2011,7,3,04
596707,"Maricopa County, AZ",04013,2011-07-04,6.910483,-2.588459,17.277122,2011,7,4,04
599940,"Maricopa County, AZ",04013,2011-07-05,6.910483,-2.588459,17.277122,2011,7,5,04
603173,"Maricopa County, AZ",04013,2011-07-06,6.910483,-2.588459,17.277122,2011,7,6,04
606406,"Maricopa County, AZ",04013,2011-07-07,6.910483,-2.588459,17.277122,2011,7,7,04
609639,"Maricopa County, AZ",04013,2011-07-08,6.910483,-2.588459,17.277122,2011,7,8,04
612872,"Maricopa County, AZ",04013,2011-07-09,6.910483,-2.588459,17.277122,2011,7,9,04
616105,"Maricopa County, AZ",04013,2011-07-10,6.910483,-2.588459,17.277122,2011,7,10,04


In [73]:
arizona_2011

,county,fips,date,AT_mean,AT_min,AT_max,year,month,day,state
585251,"Apache County, AZ",04001,2010-07-01,17.030502,9.011887,23.879784,2010,7,1,04
588484,"Apache County, AZ",04001,2010-07-02,18.649249,13.175018,26.793221,2010,7,2,04
591717,"Apache County, AZ",04001,2010-07-03,18.010435,10.375458,24.198036,2010,7,3,04
594950,"Apache County, AZ",04001,2010-07-04,16.093841,9.774536,22.953903,2010,7,4,04
598183,"Apache County, AZ",04001,2010-07-05,14.998361,2.187210,22.849701,2010,7,5,04
601416,"Apache County, AZ",04001,2010-07-06,14.502719,1.296860,26.130775,2010,7,6,04
604649,"Apache County, AZ",04001,2010-07-07,18.585924,7.267761,25.264732,2010,7,7,04
607882,"Apache County, AZ",04001,2010-07-08,18.859768,13.561905,23.613846,2010,7,8,04
611115,"Apache County, AZ",04001,2010-07-09,17.726924,10.111916,26.442017,2010,7,9,04
614348,"Apache County, AZ",04001,2010-07-10,19.052927,15.125641,26.704033,2010,7,10,04


In [74]:
arizona_2011.tail(50)

,county,fips,date,AT_mean,AT_min,AT_max,year,month,day,state
627182,"Yavapai County, AZ",04025,2010-07-13,25.169025,17.625839,29.914635,2010,7,13,04
630415,"Yavapai County, AZ",04025,2010-07-14,25.230627,18.435768,30.426346,2010,7,14,04
633648,"Yavapai County, AZ",04025,2010-07-15,26.324657,18.353310,33.650375,2010,7,15,04
636881,"Yavapai County, AZ",04025,2010-07-16,27.448091,21.533333,32.720726,2010,7,16,04
640114,"Yavapai County, AZ",04025,2010-07-17,24.126616,18.287827,32.356361,2010,7,17,04
643347,"Yavapai County, AZ",04025,2010-07-18,27.417655,20.617828,33.662781,2010,7,18,04
646580,"Yavapai County, AZ",04025,2010-07-19,26.368036,20.412575,33.395836,2010,7,19,04
649813,"Yavapai County, AZ",04025,2010-07-20,26.623613,20.737778,30.833359,2010,7,20,04
653046,"Yavapai County, AZ",04025,2010-07-21,25.637831,23.556282,29.006310,2010,7,21,04
656279,"Yavapai County, AZ",04025,2010-07-22,23.822422,18.807053,28.535805,2010,7,22,04


In [77]:
heatwave_counts.to_csv('heatwave_1960_2020/heatwave_monthly/heatwave_count_monthly_1960_2020.csv')

In [78]:
heatwave_counts['heatwave_count'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [79]:
years_30 = [year for year in range(1960, 2021)]

In [80]:
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = heatwave_counts[heatwave_counts['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [81]:
fips_numbers

[{1960: 3233},
 {1961: 3233},
 {1962: 3233},
 {1963: 3233},
 {1964: 3233},
 {1965: 3233},
 {1966: 3233},
 {1967: 3233},
 {1968: 3233},
 {1969: 3233},
 {1970: 3233},
 {1971: 3233},
 {1972: 3233},
 {1973: 3233},
 {1974: 3233},
 {1975: 3233},
 {1976: 3233},
 {1977: 3233},
 {1978: 3233},
 {1979: 3233},
 {1980: 3233},
 {1981: 3233},
 {1982: 3233},
 {1983: 3233},
 {1984: 3233},
 {1985: 3233},
 {1986: 3233},
 {1987: 3233},
 {1988: 3233},
 {1989: 3233},
 {1990: 3233},
 {1991: 3233},
 {1992: 3233},
 {1993: 3233},
 {1994: 3233},
 {1995: 3233},
 {1996: 3233},
 {1997: 3233},
 {1998: 3233},
 {1999: 3233},
 {2000: 3233},
 {2001: 3233},
 {2002: 3233},
 {2003: 3233},
 {2004: 3233},
 {2005: 3233},
 {2006: 3233},
 {2007: 3233},
 {2008: 3233},
 {2009: 3233},
 {2010: 3233},
 {2011: 3233},
 {2012: 3233},
 {2013: 3233},
 {2014: 3233},
 {2015: 3233},
 {2016: 3233},
 {2017: 3233},
 {2018: 3233},
 {2019: 3233},
 {2020: 3233}]

In [82]:
heatwave_counts_nonzero = heatwave_counts[heatwave_counts['heatwave_count'] != 0]
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = heatwave_counts_nonzero[heatwave_counts_nonzero['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [83]:
fips_numbers

[{1960: 65},
 {1961: 69},
 {1962: 57},
 {1963: 67},
 {1964: 95},
 {1965: 61},
 {1966: 170},
 {1967: 38},
 {1968: 221},
 {1969: 188},
 {1970: 120},
 {1971: 63},
 {1972: 118},
 {1973: 178},
 {1974: 53},
 {1975: 105},
 {1976: 96},
 {1977: 245},
 {1978: 255},
 {1979: 152},
 {1980: 363},
 {1981: 278},
 {1982: 125},
 {1983: 366},
 {1984: 66},
 {1985: 97},
 {1986: 245},
 {1987: 282},
 {1988: 355},
 {1989: 172},
 {1990: 130},
 {1991: 199},
 {1992: 97},
 {1993: 261},
 {1994: 111},
 {1995: 411},
 {1996: 111},
 {1997: 249},
 {1998: 358},
 {1999: 688},
 {2000: 212},
 {2001: 466},
 {2002: 447},
 {2003: 247},
 {2004: 189},
 {2005: 676},
 {2006: 639},
 {2007: 614},
 {2008: 198},
 {2009: 287},
 {2010: 841},
 {2011: 1016},
 {2012: 533},
 {2013: 527},
 {2014: 196},
 {2015: 453},
 {2016: 756},
 {2017: 417},
 {2018: 641},
 {2019: 743},
 {2020: 491}]

In [84]:
heatwave_counts_nonzero

,year,month,fips,heatwave_count
16188,1960,6,01047,1
16489,1960,6,12009,1
16500,1960,6,12033,1
17303,1960,6,22051,2
17321,1960,6,22087,1
...,...,...,...,...
2357237,2020,10,12121,1
2357238,2020,10,12123,1
2357240,2020,10,12127,1
2357406,2020,10,15007,1


In [86]:
heatwave_counts.shape

(2366556, 4)

In [87]:
heatwave_counts_nonzero['month'].unique()

array([ 6,  7,  8,  9, 10,  1,  2,  3,  4,  5, 11, 12])

In [62]:
heatwave_counts_nonzero.to_csv('heatwave_1960_2020/heatwave_count_1960_2020.csv')

In [63]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and has columns 'city', 'date', 'temperature'
# Assuming percentile_dict is your dictionary with 85th percentile temperatures for each city

# Sort by city and date
df = df.sort_values(['fips', 'month', 'date'])

In [64]:
years = [year for year in range(1960, 2021)]

In [65]:
# Define a function to calculate heatwave durations for each city
def calculate_heatwave(df, years):
    pattern = []
    for year in years:
        # Create a boolean series where True indicates a day is part of a heatwave
        city_df = df[df['year'] == year] 
        for fips in city_df['fips'].unique():
            df_fips = city_df[city_df['fips'] == fips]
            percentile = df_fips['mean_temp_p85']
            is_heatwave = df_fips['AT_min'] > percentile

            # Identify where True values change to False and vice versa,
            # and assign each group of consecutive True values a unique identifier
            heatwave_id = (is_heatwave.diff() != 0).cumsum()

            # Group by the identifiers and count the size of each group
            heatwave_lengths = heatwave_id[is_heatwave].value_counts()

            # Filter out heatwaves that are less than 2 days long
            heatwave_lengths = heatwave_lengths[heatwave_lengths >= 2]

            # Create a DataFrame with the start date of each heatwave and its duration
            heatwaves = pd.DataFrame({
                'fips': fips,
                'duration': heatwave_lengths.values,
                'year': year
            })
            pattern.append(heatwaves)
    
    return pattern

In [66]:
pattern = calculate_heatwave(df, years)
heatwave_durations = pd.concat(pattern)
heatwave_durations

,fips,duration,year
0,01045,2,1960
0,01047,5,1960
1,01047,3,1960
2,01047,2,1960
3,01047,2,1960
...,...,...,...
0,55025,2,2020
0,55029,2,2020
0,55043,2,2020
0,55051,2,2020


In [67]:
# Function to calculate heatwave durations
def calculate_heatwave_durations(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Filter for groups that are at least two days long, and return their lengths
    heatwave_durations = group_lengths[group_lengths >= 2]

    return heatwave_durations

In [68]:
# Apply the function to each year and fips
heatwave_durations = df.groupby(['year', 'fips']).apply(lambda group: calculate_heatwave_durations(group['AT_min'], group['mean_temp_p85'].iloc[0]))
# This will result in a series where each element is a series of heat wave durations
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_durations = heatwave_durations.explode().reset_index(name='heatwave_duration')
heatwave_durations

,year,fips,AT_min,heatwave_duration
0,1960,01045,6,2
1,1960,01047,4,5
2,1960,01047,18,3
3,1960,01047,8,2
4,1960,01047,10,2
...,...,...,...,...
41738,2020,55025,8,2
41739,2020,55029,2,2
41740,2020,55043,8,2
41741,2020,55051,4,2


In [69]:
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = heatwave_durations[heatwave_durations['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [70]:
fips_numbers

[{1960: 65},
 {1961: 70},
 {1962: 57},
 {1963: 70},
 {1964: 96},
 {1965: 61},
 {1966: 170},
 {1967: 38},
 {1968: 221},
 {1969: 192},
 {1970: 148},
 {1971: 64},
 {1972: 118},
 {1973: 179},
 {1974: 53},
 {1975: 105},
 {1976: 96},
 {1977: 245},
 {1978: 256},
 {1979: 155},
 {1980: 367},
 {1981: 278},
 {1982: 127},
 {1983: 367},
 {1984: 66},
 {1985: 97},
 {1986: 246},
 {1987: 286},
 {1988: 355},
 {1989: 172},
 {1990: 134},
 {1991: 203},
 {1992: 97},
 {1993: 261},
 {1994: 111},
 {1995: 413},
 {1996: 112},
 {1997: 249},
 {1998: 360},
 {1999: 709},
 {2000: 217},
 {2001: 468},
 {2002: 458},
 {2003: 248},
 {2004: 197},
 {2005: 677},
 {2006: 647},
 {2007: 614},
 {2008: 201},
 {2009: 287},
 {2010: 843},
 {2011: 1017},
 {2012: 542},
 {2013: 531},
 {2014: 196},
 {2015: 453},
 {2016: 756},
 {2017: 417},
 {2018: 656},
 {2019: 743},
 {2020: 504}]

In [71]:
heatwave_durations.to_csv('heatwave_1960_2020/heatwave_duration_1960_2020.csv')

In [88]:
df['date'] = pd.to_datetime(df['date'])

In [89]:
# Function to calculate heatwave dates
def calculate_heatwave_dates(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()
    
    # Get the dates of each group
    group_dates = series.loc[is_above_threshold, 'date'].groupby(heatwave_groups)

    # Filter for groups that are at least two days long, and return their dates
    heatwave_dates = group_dates.filter(lambda dates: len(dates) >= 2)

    return heatwave_dates

In [ ]:
# Apply the function to each year and fips
heatwave_dates = df.groupby(['year', 'month', 'fips']).apply(lambda group: calculate_heatwave_dates(group[['date', 'AT_min']], group['mean_temp_p85'].iloc[0]))
heatwave_dates

In [75]:
# The season length for each year and fips is the difference between the max and min date
season_lengths = heatwave_dates.groupby(level=['year','month','fips']).apply(lambda dates: dates.max() - dates.min())
season_lengths = season_lengths.explode().reset_index(name='heatwave_season_length')
season_lengths

,year,fips,heatwave_season_length
0,1960,01045,1 days
1,1960,01047,41 days
2,1960,01061,1 days
3,1960,01113,21 days
4,1960,05045,25 days
...,...,...,...
18136,2020,55025,1 days
18137,2020,55029,1 days
18138,2020,55043,1 days
18139,2020,55051,1 days


In [76]:
def findDaysNumber(days):
    days = str(days)
    return int(days.split()[0])+1

In [77]:
season_lengths['heatwave_length'] = season_lengths['heatwave_season_length'].apply(findDaysNumber)
season_lengths

,year,fips,heatwave_season_length,heatwave_length
0,1960,01045,1 days,2
1,1960,01047,41 days,42
2,1960,01061,1 days,2
3,1960,01113,21 days,22
4,1960,05045,25 days,26
...,...,...,...,...
18136,2020,55025,1 days,2
18137,2020,55029,1 days,2
18138,2020,55043,1 days,2
18139,2020,55051,1 days,2


In [78]:
season_lengths = season_lengths.drop('heatwave_season_length', axis = 1)
season_lengths = season_lengths.rename(columns = {'heatwave_length': 'heatwave_season_length'})
season_lengths

,year,fips,heatwave_season_length
0,1960,01045,2
1,1960,01047,42
2,1960,01061,2
3,1960,01113,22
4,1960,05045,26
...,...,...,...
18136,2020,55025,2
18137,2020,55029,2
18138,2020,55043,2
18139,2020,55051,2


In [79]:
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = season_lengths[season_lengths['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [80]:
fips_numbers

[{1960: 65},
 {1961: 70},
 {1962: 57},
 {1963: 70},
 {1964: 96},
 {1965: 61},
 {1966: 170},
 {1967: 38},
 {1968: 221},
 {1969: 192},
 {1970: 148},
 {1971: 64},
 {1972: 118},
 {1973: 179},
 {1974: 53},
 {1975: 105},
 {1976: 96},
 {1977: 245},
 {1978: 256},
 {1979: 155},
 {1980: 367},
 {1981: 278},
 {1982: 127},
 {1983: 367},
 {1984: 66},
 {1985: 97},
 {1986: 246},
 {1987: 286},
 {1988: 355},
 {1989: 172},
 {1990: 134},
 {1991: 203},
 {1992: 97},
 {1993: 261},
 {1994: 111},
 {1995: 413},
 {1996: 112},
 {1997: 249},
 {1998: 360},
 {1999: 709},
 {2000: 217},
 {2001: 468},
 {2002: 458},
 {2003: 248},
 {2004: 197},
 {2005: 677},
 {2006: 647},
 {2007: 614},
 {2008: 201},
 {2009: 287},
 {2010: 843},
 {2011: 1017},
 {2012: 542},
 {2013: 531},
 {2014: 196},
 {2015: 453},
 {2016: 756},
 {2017: 417},
 {2018: 656},
 {2019: 743},
 {2020: 504}]

In [81]:
season_lengths.to_csv('heatwave_1960_2020/heatwave_season_length_1960_2020.csv')

In [82]:
def celsius_to_fahrenheit(c):
    f = c * 9/5 + 32
    return f

In [83]:
df['AT_min'] = df['AT_min'].apply(celsius_to_fahrenheit)
df['mean_temp_p85'] = df['mean_temp_p85'].apply(celsius_to_fahrenheit)

In [75]:
# Function to calculate average exceeded temperatures above threshold during heatwaves
def calculate_heatwave_exceeded_temp_avgs(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Get the mean of exceeded temperatures (temperature - threshold) above threshold in each group
    group_exceeded_temp_avgs = (series.loc[is_above_threshold, 'AT_min'] - threshold).groupby(heatwave_groups).mean()

    # Filter for groups that are at least two days long, and return their exceeded temperature averages
    heatwave_exceeded_temp_avgs = group_exceeded_temp_avgs[group_exceeded_temp_avgs.index.map(is_above_threshold.groupby(heatwave_groups).size() >= 2)]

    return heatwave_exceeded_temp_avgs

In [76]:
# Apply the function to each year and fips
heatwave_exceeded_temp_avgs = df.groupby(['year', 'fips']).apply(lambda group: calculate_heatwave_exceeded_temp_avgs(group[['AT_min']], group['mean_temp_p85'].iloc[0]))
heatwave_exceeded_temp_avgs 

year  fips   AT_min
1960  01045  6         1.126089
      01047  4         2.826390
             8         0.267139
             10        3.829449
             14        1.010165
                         ...   
2020  55025  8         0.621146
      55029  2         1.515584
      55043  8         0.777385
      55051  4         2.759159
      55061  2         1.067584
Name: AT_min, Length: 41836, dtype: float64

In [77]:
# This will result in a series where each element is a series of heat wave exceeded temperature averages
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.explode().reset_index(name='heatwave_exceeded_temp_avg')
heatwave_exceeded_temp_avgs

,year,fips,AT_min,heatwave_exceeded_temp_avg
0,1960,01045,6,1.126089
1,1960,01047,4,2.826390
2,1960,01047,8,0.267139
3,1960,01047,10,3.829449
4,1960,01047,14,1.010165
...,...,...,...,...
41831,2020,55025,8,0.621146
41832,2020,55029,2,1.515584
41833,2020,55043,8,0.777385
41834,2020,55051,4,2.759159


In [78]:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.drop('AT_min', axis = 1)
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.rename(columns = {'heatwave_exceeded_temp_avg': 'heatwave_intensity'})
heatwave_exceeded_temp_avgs

,year,fips,heatwave_intensity
0,1960,01045,1.126089
1,1960,01047,2.826390
2,1960,01047,0.267139
3,1960,01047,3.829449
4,1960,01047,1.010165
...,...,...,...
41831,2020,55025,0.621146
41832,2020,55029,1.515584
41833,2020,55043,0.777385
41834,2020,55051,2.759159


In [79]:
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = heatwave_exceeded_temp_avgs[heatwave_exceeded_temp_avgs['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [80]:
fips_numbers

[{1960: 67},
 {1961: 70},
 {1962: 57},
 {1963: 70},
 {1964: 96},
 {1965: 61},
 {1966: 170},
 {1967: 38},
 {1968: 222},
 {1969: 192},
 {1970: 148},
 {1971: 65},
 {1972: 118},
 {1973: 179},
 {1974: 53},
 {1975: 105},
 {1976: 96},
 {1977: 245},
 {1978: 263},
 {1979: 160},
 {1980: 376},
 {1981: 278},
 {1982: 128},
 {1983: 373},
 {1984: 66},
 {1985: 97},
 {1986: 250},
 {1987: 291},
 {1988: 358},
 {1989: 173},
 {1990: 134},
 {1991: 204},
 {1992: 97},
 {1993: 263},
 {1994: 111},
 {1995: 414},
 {1996: 115},
 {1997: 253},
 {1998: 360},
 {1999: 712},
 {2000: 217},
 {2001: 469},
 {2002: 468},
 {2003: 253},
 {2004: 206},
 {2005: 681},
 {2006: 658},
 {2007: 617},
 {2008: 204},
 {2009: 289},
 {2010: 845},
 {2011: 1017},
 {2012: 542},
 {2013: 534},
 {2014: 196},
 {2015: 453},
 {2016: 757},
 {2017: 419},
 {2018: 660},
 {2019: 748},
 {2020: 504}]

In [81]:
heatwave_exceeded_temp_avgs.to_csv('heatwave_1960_2020/heatwave_intensity_1960_2020.csv')